In [ ]:
import os
import numpy as np
from PIL import Image

mlf_data = np.load(f'dataset/multispectral-light-field/Dots.npy')

out_folder = f'outputs/jpeg_dots'
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for u in range(9):
    for v in range(9):
        for c in range(13):
            mlf_spectral = mlf_data[u,v,...,c]
            image_pil = Image.fromarray(mlf_spectral, 'I;16')
            image_pil.mode = 'I'
            image_pil.point(lambda i:i*(1./256)).convert('L').save(f'{out_folder}/{u}_{v}_{c}.jpg')

In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM

mlf_data = np.load(f'dataset/multispectral-light-field/Dots.npy')

psnrs = []
ssims = []
for u in range(9):
    for v in range(9):
        for c in range(13):
            mlf_spectral = mlf_data[u,v,...,c]
            im = np.array(Image.open(f'outputs/jpeg_dots/{u}_{v}_{c}.jpg')).astype(np.uint16) * 256
            psnrs.append(PSNR(mlf_spectral, im))
            ssims.append(SSIM(mlf_spectral, im))
            
print(np.array(psnrs).mean(), np.array(ssims).mean())

In [ ]:
import os
import cv2
import numpy as np
import moviepy.video.io.ImageSequenceClip
from moviepy.editor import VideoFileClip
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM

time = 31
channels = 33

for codec in ['libx264']:
    out_folder = f'outputs/moviepy/{codec}'
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    for ci in range(channels):
        vid_imgs = []
        for ti in range(time):
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            vid_imgs.append(im)
        clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(vid_imgs, fps=24)
        clip.write_videofile(f"{out_folder}/{ci+1}.mp4", audio=False, preset='veryslow', logger=None, codec=codec, bitrate='200k')
    psnrs = []
    ssims = []
    for ci in range(channels):
        clip = VideoFileClip(f"{out_folder}/{ci+1}.mp4")
        for ti, frames in enumerate(clip.iter_frames()):
            if ti >= time:
                continue
            gray_frames = cv2.cvtColor(frames, cv2.COLOR_RGB2GRAY)
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            psnrs.append(PSNR(gray_frames, im))
            ssims.append(SSIM(gray_frames, im))
    with open(f"{out_folder}/metrics.txt",'w') as f:
        f.write(f"MEAN PSNR:{np.array(psnrs).mean()}\n")
        f.write(f"MEAN SSIM:{np.array(ssims).mean()}\n")

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# mean_psnrs4 = [25.70, 26.97, 28.02]
# mean_ssims4 = [0.71, 0.74, 0.78]
# alpha = [0, 1, 2]

# mean_psnrs5 = [26.68, 27.51, 28.10]
# mean_ssims5 = [0.73, 0.76, 0.78]

# mean_psnrs6 = [27.26, 27.83, 27.77]
# mean_ssims6 = [0.75, 0.77, 0.77]

# mean_psnrs7 = [27.61, 27.66, 26.91]
# mean_ssims7 = [0.76, 0.77, 0.74]

# sns.set()
# fig, ax2 = plt.subplots(1, 1, facecolor='w')
# ax2.set_xlabel(r'$\alpha$')
# ax = ax2.twinx()
# ax.plot(alpha, mean_psnrs4, '-', linewidth=2.5, color="lightcoral", label=r'$L=4 (PSNR)$')
# ax2.plot(alpha, mean_ssims4, '--', linewidth=2.5, color="lightcoral", label=r'$L=4 (SSIM)$')
# ax.plot(alpha, mean_psnrs5, linewidth=2.5, color="forestgreen", label=r'$L=5 (PSNR)$')
# ax2.plot(alpha, mean_ssims5, '--', linewidth=2.5, color="forestgreen", label=r'$L=5 (SSIM)$')
# ax.plot(alpha, mean_psnrs6, linewidth=2.5, color="dodgerblue", label=r'$L=6 (PSNR)$')
# ax2.plot(alpha, mean_ssims6, '--', linewidth=2.5, color="dodgerblue", label=r'$L=6 (SSIM)$')
# ax.plot(alpha, mean_psnrs7, linewidth=2.5, color="indigo", label=r'$L=7 (PSNR)$')
# ax2.plot(alpha, mean_ssims7, '--', linewidth=2.5, color="indigo", label=r'$L=7 (SSIM)$')
# ax.set_ylabel("Mean PSNR")
# ax2.set_ylabel("Mean SSIM")
# handles, labels = ax.get_legend_handles_labels()
# handles2, labels2 = ax2.get_legend_handles_labels()
# handles += handles2
# labels += labels2
# ax.legend(handles, labels, loc='lower right', ncol=2, prop={'size': 11})
# fig.tight_layout()
# plt.savefig('outputs/alpha.pdf')

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# mean_psnrs0 = [25.70, 26.68, 27.26, 27.61]
# mean_ssims0 = [0.71, 0.73, 0.75, 0.76]

# mean_psnrs1 = [26.97, 27.51, 27.83, 27.66]
# mean_ssims1 = [0.74, 0.76, 0.77, 0.77]

# mean_psnrs2 = [28.02, 28.10, 27.77, 26.91]
# mean_ssims2 = [0.78, 0.78, 0.77, 0.74]

# l = [4, 5, 6, 7]

# sns.set()
# fig, ax2 = plt.subplots(1, 1, facecolor='w')
# ax2.set_xlabel(r'L')
# ax = ax2.twinx()
# ax.plot(l, mean_psnrs0, '-', linewidth=2.5, color="orange", label=r'$\alpha=0 (PSNR)$')
# ax2.plot(l, mean_ssims0, '--', linewidth=2.5, color="orange", label=r'$\alpha=0 (SSIM)$')
# ax.plot(l, mean_psnrs1, linewidth=2.5, color="slategray", label=r'$\alpha=1 (PSNR)$')
# ax2.plot(l, mean_ssims1, '--', linewidth=2.5, color="slategray", label=r'$\alpha=1 (SSIM)$')
# ax.plot(l, mean_psnrs2, linewidth=2.5, color="darkmagenta", label=r'$\alpha=2 (PSNR)$')
# ax2.plot(l, mean_ssims2, '--', linewidth=2.5, color="darkmagenta", label=r'$\alpha=2 (SSIM)$')
# ax.set_ylabel("Mean PSNR")
# ax2.set_ylabel("Mean SSIM")
# handles, labels = ax.get_legend_handles_labels()
# handles2, labels2 = ax2.get_legend_handles_labels()
# handles += handles2
# labels += labels2
# ax.legend(handles, labels, loc='lower right', ncol=2, prop={'size': 11})
# fig.tight_layout()
# plt.savefig('outputs/l.pdf')

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# mean_psnrs1 = [31.96, 35.20, 38.58, 39.96, 40.44]
# mean_ssims1 = [0.87, 0.91, 0.94, 0.95, 0.95]
# compressed_ratio1 = [100 * (1 - 275175 / 180338675),
#                      100 * (1 - 438066 / 180338675),
#                      100 * (1 - 1479375 / 180338675),
#                      100 * (1 - 2279356 / 180338675),
#                      100 * (1 - 3560501 / 180338675),
#                     ]

# mean_psnrs2 = [30.58, 38.15, 38.15]
# mean_ssims2 = [0.88, 0.92, 0.92]
# compressed_ratio2 = [100 * (1 - 279303 / 180338675),
#                      100 * (1 - 3230919 / 180338675),
#                      100 * (1 - 3560501 / 180338675),
#                     ]

# sns.set()
# fig, ax2 = plt.subplots(1, 1, facecolor='w')
# ax = ax2.twinx()
# ax.plot(compressed_ratio1, mean_psnrs1, '-', linewidth=2.5, color="maroon", label="Video Encoding (PSNR)")
# ax2.plot(compressed_ratio1, mean_ssims1, '--', linewidth=2.5, color="maroon", label="Video Encoding (SSIM)")
# ax.plot(compressed_ratio2, mean_psnrs2, linewidth=2.5, color="lightcoral", label="SIREN (PSNR)")
# ax2.plot(compressed_ratio2, mean_ssims2, '--', linewidth=2.5, color="lightcoral", label="SIREN (SSIM)")
# # ax.set_ylim(top=41.8)
# # ax2.set_ylim(top=0.97)
# ax.set_ylabel("Mean PSNR")
# ax2.set_ylabel("Mean SSIM")
# ax.set_xlabel(r"Compression Rate")
# handles, labels = ax.get_legend_handles_labels()
# handles2, labels2 = ax2.get_legend_handles_labels()
# handles += handles2
# labels += labels2
# ax.legend(handles, labels, loc='upper right', ncol=1, prop={'size': 11})
# fig.tight_layout()
# plt.savefig('outputs/baseline.pdf')